# Neural Networks Sprint Challenge

## 1) Define the following terms:

- Neuron
- Input Layer
- Hidden Layer
- Output Layer
- Activation
- Backpropagation

 YOUR ANSWER HERE

## 2) Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 1  | 1  | 1  | 1 |
| 1  | 0  | 1  | 0 |
| 0  | 1  | 1  | 0 |
| 0  | 0  | 1  | 0 |

In [25]:
import numpy as np
import pandas as pd
import category_encoders as ce
import keras
from pandarallel import pandarallel
pandarallel.initialize()

def squish(x: np.number) -> np.number: 
    return np.divide(1, 1 + np.exp(-x))

def del_squish(x: np.number) -> np.number: 
    s = squish(x)
    return s * (1 - s)


New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 4 workers


In [26]:
X = np.array([[1,1,1], [1,0,1], [0,1,1], [0,0,1]])
y = np.array([[1],[0],[0],[0]])

class Perceptron: 
    def __init__(self, X, y): 
        self.X = X
        self.y = y
        self.inputs = X.shape[1]
        self.weights = np.array([1,1,-1])
        self.prediction = [self.relu(x) for x in self.X @ self.weights]
    
    def relu(self, x): 
        if x < 0: 
            return 0
        else: 
            return x

P = Perceptron(X, y)

P.prediction

[1, 0, 0, 0]

## 3) Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. 
- Your network must have one hidden layer. 
- You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
- Train your model on the Heart Disease dataset from UCI:

[Github Dataset](https://github.com/ryanleeallred/datasets/blob/master/heart.csv)

[Raw File on Github](https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv)


In [44]:
def clean(dat): 
    assert dat.isna().sum().sum()==0
    assert all([t.name in ['int64', 'float64'] for t in dat.dtypes])
    print(dat.shape)
    return (dat.drop('target', axis=1),
           dat.target)

X, y = clean(pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv"))

X.head()

(303, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [84]:
class NeuralNetwork: 
    def __init__(self, X, y): 
        self.X = X.values
        self.y = y.values.reshape(-1,1)
        self.inputs = X.shape[1]
        self.hidden_1 = X.shape[1]
        self.output_nodes = 1
        # init weights: 
        self.L1_weights = np.random.randn(self.inputs, self.hidden_1)
        self.L2_weights = np.random.randn(self.hidden_1, self.output_nodes)
        self.predictions = self.refresh_ff()
        
    def feed_forward(self): 
        ''' matmul to produce predictions ''' 
        hidden_sum_1 = self.X @ self.L1_weights
        self.activated_hidden_1 = squish(hidden_sum_1)
        hidden_sum_2 = self.activated_hidden_1 @ self.L2_weights
        self.activated_hidden_2 = squish(hidden_sum_2)
        return self.activated_hidden_2

    def refresh_ff(self): 
        ''' run this when weights are updated '''
        return self.feed_forward()
    
    def loss(self): 
        ''' mean squared error '''
        n = len(self.y)
        assert len(self.predictions)==n
        return np.divide(sum([Y**2 for Y in self.predictions - self.y]), n)
    
    def back(self): 
        ''' will modify values of Lk_weights '''
        self.predictions = self.refresh_ff()
        output_error = self.predictions - self.y
        del_output_error = output_error * del_squish(self.predictions) # ??????? 

        s2_error = del_output_error @ self.L2_weights.T
        del_s2_error = s2_error * del_squish(self.activated_hidden_2)
        
        s1_error = del_s2_error @ self.L1_weights.T
        del_s1_error = s1_error * del_squish(self.activated_hidden_1)
        
        assert self.L1_weights.shape == (X.T @ del_s2_error).shape
        self.L1_weights = X.T @ del_s2_error
        self.L2_weights = self.activated_hidden_1.T @ del_output_error
        pass
    
NN = NeuralNetwork(X,y)

ls = []
for i in range(50): 
    NN.back()
    if i%11==0: 
        ls.append(NN.loss())

print(ls)

[array([0.30359702]), 0.0, 0.0, 0.0, 0.0]


## 4) Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy. 

- Use the Heart Disease Dataset (binary classification)
- Use an appropriate loss function for a binary classification task
- Use an appropriate activation function on the final layer of your network. 
- Train your model using verbose output for ease of grading.
- Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
- When hyperparameter tuning, show you work by adding code cells for each new experiment. 
- Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
- You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [ ]:
##### Your Code Here #####